In [1]:
import torch
import pandas as pd
import numpy as np
from numpy.random import default_rng

In [2]:
# Loads a matrix from a CSV file, skipping the first line and compressing numbers to float
def load_matrix(file_path):
    return pd.read_csv(file_path, header=None, skiprows=1).values.astype(float)

# Computes the weighted adyacency matrix W based on distance
def calculate_adjacency_matrix(distances, sigma_squared=10, epsilon=0.5):
    """
    Computes the weighted adyacency matrix W based on distance
    Args:
    distances (np.ndarray): Distance matrix with shape (N, N).
    sigma_squared (float): Value of sigma squared to compute weights.
    epsilon (float): Threshold to determine if a weight is enough to be included in the matrix.
    Returns:
    np.ndarray: The weighted adyacency matrix W.
    """
    N = distances.shape[0]
    W = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            if i != j:
                weight = np.exp(-distances[i, j]**2 / sigma_squared)
                if weight >= epsilon:
                    W[i, j] = weight

    return W

import torch
import numpy as np

# Computes scaled Laplacian of an adjacency matrix
def scaled_laplacian(A):
    """
    Computes scaled Laplacian of an adjacency matrix.

    Args:
    A (numpy.ndarray): Adjacency matrix.

    Returns:
    numpy.ndarray: Scaled Laplacian.
    """
    n = A.shape[0]
    d = np.sum(A, axis=1) # Sum of each row (degree of each node)
    L = np.diag(d) - A # Non-scaled Laplacian

    # Scaling the Laplacian
    for i in range(n):
        for j in range(n):
            if d[i] > 0 and d[j] > 0:
                L[i, j] /= np.sqrt(d[i] * d[j])

    # Computing the max value of the eigenvalues
    lam = np.linalg.eigvals(L).max().real

    # Returning the scales Laplacian
    return 2 * L / lam - np.eye(n)

# Computes Chebyshev polynomials up to degree Ks for laplacian L
def cheb_poly(L, Ks):
    """
    Computes Chebyshev polynomials up to degree Ks for laplacian L.

    Args:
    L (numpy.ndarray): Scaled Laplacian.
    Ks (int): Max degree of Chebyshev polynomials.

    Returns:
    numpy.ndarray: The Chebyshev polynomials.
    """
    n = L.shape[0]
    LL = [np.eye(n), L[:]]  # T_0(L) = I y T_1(L) = L

    # Computing T_k(L) = 2L * T_{k-1}(L) - T_{k-2}(L)
    for i in range(2, Ks):
        LL.append(np.matmul(2 * L, LL[-1]) - LL[-2])

    return np.asarray(LL)

In [5]:
# Returns the adjacency matrix computed from raw data

# Currently, raw data are just Kms distances between countries
# The way raw data are used depends on the option that is taken about that

def get_distance_matrix(dist_matrix_dict,distances="Kms"):
    # TO-DO: consider other ways to build the matrix; for example, the total amount of import-export between two countries

    dist_matrix_path = dist_matrix_dict[distances]
    df_matrix = pd.read_csv(dist_matrix_path)
    W = load_matrix(dist_matrix_path)

    return W

def get_adjacency_matrix(option,dist_matrix_dict,distances,Ks):
    W = get_distance_matrix(dist_matrix_dict,distances)
    if option == "Laplacian":
        # Computing scaled Laplacian from the adjacency matrix
        print("Computing the scaled Laplacian")
        L = scaled_laplacian(W)
    elif option == "Reciprocal":
        # Inverting each value of W (1/x)
        print("Computing the reciprocal")
        L = np.reciprocal(W)
    elif option == "Exponential reciprocal":
        # Inverting exponentially (according to the paper)
        print("Computing exponential")
        L = calculate_adjacency_matrix(W)
    elif option == "Random":
        print("Producing random data")
        L = default_rng().random((W.shape[0],W.shape[0]))
    elif option == "Constant":
        print("Producing constant data")
        L = np.ones((W.shape[0],W.shape[0]))/2
    else: # "Raw"        
        print("Taking raw data as they are")
        L = W

    # Computing Chebyshev polynomials (Ks is the degree of the polynomials)
    Lk = cheb_poly(L, Ks)

    # Converting Chebyshev polynomials to PyTorch tensors and moving them to the device
    Lk = torch.Tensor(Lk.astype(np.float32))

    return Lk